In [46]:
import glob
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import operator 
from collections import Counter
from nltk import bigrams
import pandas as pd
import csv
import unicodecsv
import os
import glob

In [53]:
#This function returns absolute path of all the files in a directory as a dictionary
def allFileNamesInDir(dirName):
    itr = glob.glob(os.path.join(directory, '*.json'))
    dir_map={}
    for filename in itr:
        #print(re.search(r'\d{1,2}',filename).group(0))
        dir_map[int(re.search(r'\d{1,5}',filename).group(0))]=filename
    return dir_map

Loading data file names from the directory and gathering them in a map to iterate in a sorted order

In [47]:
directory = '/Users/azhar/Documents/python/data/'
dir_map = allFileNamesInDir(directory)

<h2> No need to call run the next codeblock if new lines are already removed </h2>
Removing empty lines from all the files and renaming them as 0.json, 1.json ...

In [49]:
newfile_name_format = directory+'{}'+'.json'

for i in range(len(dir_map)):
#for i in dir_map:
    with open(dir_map[i]) as infile, open(newfile_name_format.format(i), 'w') as outfile:
        for line in infile:
            if not line.strip(): continue  # skip the empty line
            outfile.write(line)  # non-empty line. Write it to output
    os.remove(dir_map[i]) # removing the old file

In [82]:
#showing all the file names in sorted order
directory = '/Volumes/Untitled/data/'
all_files = allFileNamesInDir(directory)
for i in range(len(all_files)):
    print(all_files[i])

/Volumes/Untitled/data/0.json
/Volumes/Untitled/data/1.json
/Volumes/Untitled/data/2.json
/Volumes/Untitled/data/3.json
/Volumes/Untitled/data/4.json
/Volumes/Untitled/data/5.json
/Volumes/Untitled/data/6.json
/Volumes/Untitled/data/7.json
/Volumes/Untitled/data/8.json
/Volumes/Untitled/data/9.json
/Volumes/Untitled/data/10.json
/Volumes/Untitled/data/11.json
/Volumes/Untitled/data/12.json
/Volumes/Untitled/data/13.json
/Volumes/Untitled/data/14.json
/Volumes/Untitled/data/15.json
/Volumes/Untitled/data/16.json
/Volumes/Untitled/data/17.json
/Volumes/Untitled/data/18.json
/Volumes/Untitled/data/19.json
/Volumes/Untitled/data/20.json
/Volumes/Untitled/data/21.json
/Volumes/Untitled/data/22.json
/Volumes/Untitled/data/23.json
/Volumes/Untitled/data/24.json


In [83]:
line = None
with open(all_files[0],'r') as f:
    line = f.readline()

tweet =  json.loads(line)
tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Feb 13 23:09:37 +0000 2018',
 'display_text_range': [15, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/963550760012271616',
    'indices': [117, 140],
    'url': 'https://t.co/VX6ALHjESH'}],
  'user_mentions': [{'id': 1850835966,
    'id_str': '1850835966',
    'indices': [0, 14],
    'name': 'Criptominers',
    'screen_name': 'cripto_miners'}]},
 'extended_tweet': {'display_text_range': [15, 268],
  'entities': {'hashtags': [],
   'symbols': [],
   'urls': [],
   'user_mentions': [{'id': 1850835966,
     'id_str': '1850835966',
     'indices': [0, 14],
     'name': 'Criptominers',
     'screen_name': 'cripto_miners'}]},
  'full_text': '@cripto_miners Bitcoin Gold (BTG) is gaining popularity and becoming more expensive, but you still don’t own it? You have a unique chance to profit and exchange without co

In [80]:
lines = None
with open(all_files[0],'r') as f:
    lines = f.readlines()

maximum = lines[0]
for line in lines:
    if len(json.loads(line) > json.loads(maximum)):
        

In [123]:
#Finding all the files in directory as a list

csv_dir = os.path.join(directory, 'csv/')
output_csv_file_name_format = csv_dir +'{}'+'.csv'

onlyfiles = allFileNamesInDir(directory)

#read the files and insert tweets into tweetsList
for t in range(0,1):
    f = open(onlyfiles[t], "r")
    lines = f.readlines()
    f.close
    x=0
    tweetsList = []
    for line in lines:
        try:
            tweet = json.loads(line)
            tweetsList.append(tweet)
            x= x+1;
        except Exception as e:
            continue
    #creating data frame
    tweets = pd.DataFrame(tweetsList)
    #print(len(tweets))
    #print(tweets.head(1))

    #droping limit columns created due to empty lines
    if 'limit' in tweets.columns:
        tweets = tweets.drop(['limit'], axis=1)

    # dropping rows will all nan
    tweets = tweets.dropna(axis=0, how='all')

    #Adding user info to data frame columns
    l1 = ['user_id', 'user_created_at', 'user_followers_count', 'user_friends_count', 'user_location', 'user_screen_name', 'user_verified', 'user_description']
    l2 = ['id', 'created_at', 'followers_count', 'friends_count', 'location', 'screen_name', 'verified', 'description']
    for i in range(0, len(l1)):
        tweets[l1[i]] = list(map(lambda tweet: tweet[l2[i]] if tweet!= None else None, tweets['user'].tolist()))

    #Adding location info to data frame columns
    l3 = ['place_bounding_box', 'place_country', 'place_country_code', 'place_full_name', 'place_name', 'place_place_type']
    l4 = ['bounding_box', 'country', 'country_code', 'full_name', 'name', 'place_type']
    for i in range(0, len(l3)):
        tweets[l3[i]] = list(map(lambda tweet: tweet[l4[i]] if tweet!= None else None, tweets['place'].tolist()))

#    tweets.head()
    print('writing to output file = '+output_csv_file_name_format.format(t))
    print('total columns = '+ str(len(tweets.columns)))
    print('total rows = '+ str(len(tweets)))
    print('--------------------------------------------------------------------------------------')
    tweets.index = range(len(tweets.index))
    tweets.to_pickle(output_csv_file_name_format.format(t))

writing to output file = /Volumes/Untitled/data/csv/0.csv
total columns = 49
total rows = 7506
--------------------------------------------------------------------------------------


In [121]:
tweets['place_bounding_box'].sample(10)

6508                                                 None
4695                                                 None
7012                                                 None
6532                                                 None
4533                                                 None
4189    {'type': 'Polygon', 'coordinates': [[[115.6176...
7105                                                 None
1787                                                 None
5597                                                 None
3203                                                 None
Name: place_bounding_box, dtype: object

In [87]:
lines = None
with open(all_files[0],'r') as f:
    lines = f.readlines()

t = json.loads(lines[0])
t

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Feb 13 23:09:37 +0000 2018',
 'display_text_range': [15, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/963550760012271616',
    'indices': [117, 140],
    'url': 'https://t.co/VX6ALHjESH'}],
  'user_mentions': [{'id': 1850835966,
    'id_str': '1850835966',
    'indices': [0, 14],
    'name': 'Criptominers',
    'screen_name': 'cripto_miners'}]},
 'extended_tweet': {'display_text_range': [15, 268],
  'entities': {'hashtags': [],
   'symbols': [],
   'urls': [],
   'user_mentions': [{'id': 1850835966,
     'id_str': '1850835966',
     'indices': [0, 14],
     'name': 'Criptominers',
     'screen_name': 'cripto_miners'}]},
  'full_text': '@cripto_miners Bitcoin Gold (BTG) is gaining popularity and becoming more expensive, but you still don’t own it? You have a unique chance to profit and exchange without co

In [91]:
len(lines)

7507

In [23]:
# Removing unnecessary words and characters
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + [':/','com',':D','RT','rt', 'via', 'RT', 'I', 'We', 'After', 'The', 'the', '...', 'amp', 'Our', 'accept', 'A', 'FL', 'krab','https','@aoojadi','took','This','us','like','You','put','bathtub','…']

tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP I ...'
print(preprocess(tweet))

terms_all = [term for term in preprocess(tweet) if term not in stop]
print(terms_all)

['marcobonzanini', 'just', 'example', ':D', 'http', ':/', 'example', 'com', 'NLP']
['marcobonzanini', 'example', 'http', 'example', 'NLP']


In [8]:
f = open("crypto_tweetsNoL.json", "r")
lines = f.readlines()
f.close
i=0
for line in lines:
    if i==38196 or 40150:
        i=i+1
        continue
    tweet = json.loads(line)
    i= i+1;
print(i)

1439


In [30]:
fname = 'crypto_tweets2NoL.json'
with open(fname, 'r') as f:
    count_all = Counter()
    count_bigram = Counter()
    i = -1
    for line in f:
        i=i+1
        if i>500000:
            break
        elif i==38196:
            continue
        elif i==40150:
            continue
        elif i>=268148:
            break
        else:
            #print(line)
            tweet = json.loads(line)
            keys = tweet.keys()
            if 'text' in tweet:
                #print(i)
                # Create a list with all the terms
                terms_all = [term for term in preprocess(tweet['text']) if term not in stop]
                terms_bigram = bigrams(terms_all)
                # Update the counter
                count_all.update(terms_all)
                count_bigram.update(terms_bigram)
    # Print the first 5 most frequent words
    print(count_all.most_common(20))
    print(count_bigram.most_common(20))

[('Bitcoin', 55635), ('BTC', 36612), ('global', 21561), ('BTG', 20950), ('cryptocurrency', 20263), ('still', 20056), ('exchanges', 19891), ('launch', 19680), ('start', 19530), ('Gold', 19365), ('Successfully', 18987), ('mainnet', 18879), ('gaining', 18723), ('popularity', 18699), ('becoming', 18662), ('trades', 18616), ('expensive', 18322), ('bitcoin', 18229), ('balance', 17967), ('crypto', 17599)]
[(('Successfully', 'launch'), 18869), (('launch', 'mainnet'), 18845), (('Bitcoin', 'Gold'), 18794), (('mainnet', 'global'), 18774), (('Gold', 'BTG'), 18734), (('global', 'exchanges'), 18725), (('BTG', 'gaining'), 18689), (('exchanges', 'start'), 18663), (('gaining', 'popularity'), 18633), (('popularity', 'becoming'), 18536), (('start', 'trades'), 18502), (('becoming', 'expensive'), 18282), (('expensive', 'still'), 17845), (('trades', 'balance'), 17836), (('balance', 'BTC'), 16060), (('claim', 'free'), 6157), (('Use', 'link'), 6118), (('ClassyCoin', 'claim'), 6040), (('free', 'Classycoin'), 6

In [15]:
with open('crypto_tweetsNoL.json') as infile, open('sample_1000_tweets.json', 'w') as outfile:
    i = 1;
    for line in infile:
        if not line.strip(): continue  # skip the empty line
        outfile.write(line)  # non-empty line. Write it to output
        if i>999:
            break
        i+=1

In [16]:
f = open("sample_1000_tweets.json", "r")
lines = f.readlines()
f.close
i=0
for line in lines:
    if i==38196 or 40150:
        i=i+1
        continue
    tweet = json.loads(line)
    i= i+1;
print(i)

1000
